<a href="https://colab.research.google.com/github/AnnSenina/python_hse_2024/blob/main/notebooks/8_%D0%9A%D0%BE%D1%80%D1%80%D0%B5%D0%BB%D1%8F%D1%86%D0%B8%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Открываем данные

Начнем с тех же самых арендованных велосипедов

Исходный датасет с [Kaggle](https://www.kaggle.com/datasets/kratos2597/boom-bikes-linear-regression/data)

Мы работаем с немного измененной версией от коллег из проекта Data Culture

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/databases_stat_2023/main/BikeDataVar.csv')
df.head()

Для некоторых видов анализа пропуски будут мешать


In [6]:
df = df.dropna() # удалить пропуски

In [ ]:
df.info()

Посчитаем все корреляции разом

In [ ]:
df.corr() # pandas по умолчанию больше не выбирает за нас числовые столбцы

In [ ]:
import numpy as np
df_num = df.select_dtypes(include=np.number)
df_num.corr()

In [ ]:
# ИЛИ:
df.corr(numeric_only = True)

In [ ]:
df[['Temperature', 'Humidity', 'Wind speed', 'Rainfall', 'Snowfall', 'Rental Count']].corr()
# давайте оставим меньшее количество столбцов, чтобы с ними было удобно работать

In [ ]:
# можно включить тепловую карту прямо в матрицу корреляций:
df2 = df[['Temperature', 'Humidity', 'Wind speed', 'Rainfall', 'Snowfall', 'Rental Count']]
df2.corr().style.background_gradient(cmap='coolwarm')

Корреляция Пирсона, Кендалла и Спирмена в pandas в настоящее время рассчитывается с использованием попарно полных наблюдений (исключая значения NA/null)

In [ ]:
# Из документации: Method of correlation:
#pearson : standard correlation coefficient
#kendall : Kendall Tau correlation coefficient
#spearman : Spearman rank correlation

# рассчитаем все 3 коэффициента, найдем различия
df2.corr('pearson') # для числовых (в идеале - непрерывных) переменных

In [ ]:
df3 = df[['Good Weather', 'Rental Count']]
df3.corr('spearman') # коэф. Спирмена считается для 1 порядковой переменной (шкала лучше - хуже) и одной количественной

In [ ]:
df4 = df[['Good Weather', 'Normal Humidity']] # в идеале должны быть 2 переменных, измеренных в порядковой шкале
df3.corr('kendall')

### p-value

К сожалению, в df.corr() по умолчанию не считается p-value

In [ ]:
# рассчитаем p-value

# документация https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html
from scipy import stats

res = stats.pearsonr(df['Temperature'], df['Humidity'])
res

In [ ]:
res.pvalue < 0.05

Аналогично:


*   https://docs.scipy.org/doc/scipy-1.13.0/reference/generated/scipy.stats.spearmanr.html
*   https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kendalltau.html



Дальше будем добавлять визуализацию

In [ ]:
# scatterplot
import matplotlib.pyplot as plt
import seaborn as sns

plt.scatter(x = df['Temperature'], y = df['Rental Count']);

In [ ]:
sns.heatmap(df2.corr());

In [ ]:
# correlogram, или тепловая карта
plt.figure(figsize=(12,10), dpi= 80) # размер картинки
sns.heatmap(df2.corr(), cmap='RdYlGn', center=0, annot=True)
plt.title('Correlogram', fontsize=22);

In [ ]:
# seaborn pairplot
# добавим переменную Good Weather (категориальная переменная позволяет задать цвет точкам по категориям)
sns.pairplot(df[['Temperature', 'Humidity', 'Wind speed', 'Rainfall', 'Snowfall', 'Rental Count', 'Good Weather']], hue='Good Weather');

### Иногда нашим данным может помочь преобразование

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.info()

In [ ]:
df['Date'].dt.month

In [ ]:
bikes_sum = df.groupby(df['Date'].dt.month)['Rental Count'].sum()
temp_median = df.groupby(df['Date'].dt.month)['Temperature'].median()
humidity_median = df.groupby(df['Date'].dt.month)['Humidity'].median()
wind_median = df.groupby(df['Date'].dt.month)['Wind speed'].median()

df_month = pd.concat([bikes_sum, temp_median, humidity_median, wind_median], axis=1)
df_month

In [ ]:
sns.heatmap(df_month.corr(), annot=True, cmap='RdYlGn', center=0)
plt.title('Weeks', fontsize=22);

In [ ]:
sns.pairplot(df_month);

Важно! Что могло пойти не так:

Временные ряды — это наборы данных, где каждая точка данных связана с определенным моментом времени

Тренд будет оказывать результаты на корреляционный анализ - и показывать ложную корреляцию ([Как избежать ошибок при работе с временными рядами](https://www.svds.com/avoiding-common-mistakes-with-time-series/))

См. подборку [безумных корреляций](https://rationalnumbers.ru/?go=all/bezumnye-korrelyacii/)

### Задания

1. Работаем с таблицей о сельскохозяйственных культурах ([Kaggle](https://www.kaggle.com/datasets/rishabhrathore055/datas/data))

*   Постройте матрицу корреляций
*   Визуализируйте в виде тепловой карты
*   Найдите наибольший коэффициент корреляции
*   Вычислите для него p-value

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/python_hse_2024/main/data/Crop_recommendation.csv')
df

Решение

In [ ]:
# @title
df = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/python_hse_2024/main/data/Crop_recommendation.csv')
df.corr(numeric_only=True)

In [ ]:
# @title
sns.heatmap(df.corr(numeric_only=True), center = 0, annot=True, cmap='RdYlGn');

In [ ]:
# @title
res = stats.pearsonr(df['temperature'], df['humidity'])
res.pvalue < 0.05

2. Работаем со старой таблицей государств-колоний



*   Постройте матрицу корреляций
*   Выберите любые 2 показателя, для которых корректно посчтитать коэффициент корреляции Спирмена; Кендалла
*   Посчитайте соответсвтующие коэффициенты и p-value
*   Возуалиируйте попарные корреляции с помощью plt.scatter()

Пирсон (по умолчнаию) - числовая + числовая
Спирмен - порядковая + числовая
Кендалл - порядковая + порядковая



In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/python_hse_2024/main/data/Colonial.csv', sep=';')
# перекодируем все переменные
df['decolonization'] = df['decolonization'].apply(lambda x: 1 if x == 'long' else 0)
df['foreign trade'] = df['foreign trade'].apply(lambda x: 0 if x == 'not appl.' else (1 if x == 'open door' else 2))
df['plantations'] = df['plantations'].apply(lambda x: 1 if x == 'little' else (2 if x == 'extensive' else 0))
df['gold/silver'] = df['gold/silver'].apply(lambda x: 1 if x == 'little' else (2 if x == 'extensive' else 0))
df = df.dropna()
df

In [ ]:
# @title

res = stats.spearmanr(df['COLYEARS'], df['violence'])
print(res)
res.pvalue < 0.05

In [ ]:
# @title
plt.scatter(df['COLYEARS'], df['violence']); # связь слабая -> практически отсутствует

In [ ]:
# @title
res = stats.kendalltau(df['violence'], df['foreign trade'])
print(res)
res.pvalue < 0.05

In [ ]:
# @title
plt.scatter(df['violence'], df['foreign trade']); # связь слабая

In [ ]:
# @title
# можно также построить всю матрицу - но будьте внимательны, не каждой паре наблюдений подходит выбранный метод рассчета корреляции

df.corr('spearman', numeric_only=True)
# df.corr('kendall', numeric_only=True)